In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

### 0. Prepare data

In [3]:
# read in all the words
words = open('../lecture2/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [12]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
... ---> a
..a ---> v
.av ---> a
ava ---> .
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [13]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

### 1. Calculations

In [27]:
# matrix with weights 
C = torch.randn([27, 2])

#### 1.1. Get weights for certain index 

In [34]:
# just get array for letter by indexing 2d array
C[5]

tensor([-2.6176,  1.5082])

In [37]:
# indexing of tensor
C[[5,6,7,7]]

tensor([[-2.6176,  1.5082],
        [-0.7809,  0.2467],
        [ 0.0140, -1.0030],
        [ 0.0140, -1.0030]])

In [35]:
# multiplying on one-hot encoded matrix 
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-2.6176,  1.5082])

#### 1.2. Construct embeddings 

In [47]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

#### 1.3. Initialize weights

In [72]:
W1 = torch.randn([6, 100])
b1 = torch.randn([100])

#### 1.4. Reshape embeddings 

In [73]:
# manually divide tensor by one of axis and concatenate
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1).shape

torch.Size([32, 6])

In [74]:
# automatically divide by axis and concatenate 
torch.cat(torch.unbind(emb, 1), dim=1).shape

torch.Size([32, 6])

In [75]:
# reshape with view. Really quick
emb.view((-1, 6)).shape

torch.Size([32, 6])

#### 1.5. Hidden state

In [76]:
h = torch.tanh(emb.view((-1, 6)) @ W1 + b1)
h.shape

torch.Size([32, 100])

In [77]:
# Checking broadcast
# 32 100 
#    100
# everything is ok

In [78]:
emb[:, 0, :].shape

torch.Size([32, 2])

#### 1.6. Final layer

In [79]:
W2 = torch.randn([100, 27])
b2 = torch.randn([27])

In [80]:
logits = h @ W2 + b2

In [83]:
counts = logits.exp()

In [90]:
prob = counts / counts.sum(axis=1, keepdims=True)
prob.shape

torch.Size([32, 27])

In [89]:
# check sum of probabilities for one letter
prob[0].sum()

tensor(1.)

#### 1.7. Loss 

In [95]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.3726)

### 2. Iteration of training process 

#### 2.1. Initialize parameters

In [102]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [103]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

#### 2.1. Forward path

In [105]:
emb = C[X]
h = torch.tanh(emb.view((-1, 6)) @ W1 + b1)
logits = h @ W2 + b2
# counts = logits.exp()
# prob = counts / counts.sum(axis=1, keepdims=True)
# loss = -prob[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits, Y)
loss

tensor(17.7697)